In [ ]:
%pip install openai langchain google-api-python-client openai"[embeddings]"

In [27]:
# Generate a dataset for products

import openai

max_products = 30
prompt = f"""
Generate a dataset for products with the following requirements:
1. Categories are Toys, Apparel, Electronics, Kitchen appliances and Home decor.
2. Each product should have an ID, Name, Description (200 characters max) and Category.
3. Toys should have an extra range of age group associated with each product info. This is an optional for all other categories.
4. The dataset must have {max_products} products.
5. The product description should be wrapped in double quotes.
6. The dataset must be in CSV format.
"""

response = openai.Completion.create(
    engine="testdavinci003",
    prompt=prompt,
    temperature=1,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
)

print(response.choices[0].text)


ID,Name,Description,Category,Age Group
1,"Action Figure","A figurine of a superhero equipped with weapons and articulation","Toys","6-8"
2,"Stuffed Animal","A soft toy made from synthetic fabric","Toys","3-5"
3,"Bicycle","A two-wheeled pedal-driven vehicle","Toys","7-11"
4,"Train Set","A set of miniature trains and tracks","Toys","7-11"
5,"Dress","A one piece garment with a skirt","Apparel","NA"
6,"Buttoned Shirt","A long-sleeved top with buttons","Apparel","NA"
7,"Laptop","A portable computer with an integrated display","Electronics","NA"
8,"Smartphone","A small personal computer with a touchscreen and phone capabilities","Electronics","NA"
9,"Toaster","A kitchen appliance used to make toast","Kitchen Appliances","NA"
10,"Blender","A device used to blend liquids","Kitchen Appliances","NA"
11,"Table Lamp","An electric powered light source for desktops","Home Decor","NA"
12,"Vase","A container for a variety of objects, usually with a tall, slender neck","Home Decor","NA"
13,"Remote Con

In [51]:
# Generate Products Embeddings

from openai.embeddings_utils import get_embedding
import pandas as pd

df = pd.read_csv("data/Products.csv", na_filter=False)

df["Combined"] = (
    "ID: "
    + df.ID.astype("str")
    + "; Name: "
    + df.Name.str.strip()
    + "; Description: "
    + df.Description.str.strip()
    + "; Category: "
    + df.Category.str.strip()
    + "; Age: "
    + df.Age.str.strip()
)

df["Embedding"] = df.Combined.apply(lambda x: get_embedding(x, engine="embedding-ada"))

df.to_csv("data/Products-Embeddings.csv", index=False)

df.head(2)

,ID,Name,Description,Category,Age,Combined,Embedding
0,1,Action Figure,A figurine of a superhero equipped with weapon...,Toys,6-8,ID: 1; Name: Action Figure; Description: A fig...,"[-0.03661048412322998, -0.004240584094077349, ..."
1,2,Stuffed Animal,A soft toy made from synthetic fabric,Toys,3-5,ID: 2; Name: Stuffed Animal; Description: A so...,"[-0.019211014732718468, 0.0013984778197482228,..."
